## Read in data

In [1]:
import pickle
import numpy as np

with open('match_list.pkl', 'rb') as f:
    match_list = pickle.load(f)
f.close()

team_hero_list = []
enemy_hero_list = []

for match in match_list:
    
    for i in match:
        if i['playerName'] == 'Wakamar':
            win = int(i['Win_Status'] == 'Winner')

    for i in match:
        if i['playerName'] != 'Wakamar':
            
            char_dict ={}
            if int(i['Win_Status'] == 'Winner') == win:
                for j in i:
                    char_dict[j] = i[j]
                team_hero_list.append(char_dict)

            if int(i['Win_Status'] == 'Winner') != win:
                for j in i:
                    char_dict[j] = i[j]
                enemy_hero_list.append(char_dict)
                
import pandas as pd

team_df = pd.DataFrame(team_hero_list)
enemy_df = pd.DataFrame(enemy_hero_list)

# Prune to only include heroes with at least 5 games as both teammate and enemy

e_pop = (enemy_df['Reference_Name'].value_counts() > 5).keys()[enemy_df['Reference_Name'].value_counts() > 5]
e_pop_set = set(e_pop)
t_pop = (team_df['Reference_Name'].value_counts() > 5).keys()[team_df['Reference_Name'].value_counts() > 5]
t_pop_set = set(t_pop)
pop = set.intersection(e_pop_set, t_pop_set)

enemy_df_pop = enemy_df.loc[enemy_df['Reference_Name'].isin(pop)]
team_df_pop = team_df.loc[team_df['Reference_Name'].isin(pop)]

## X is never on my team

In [2]:
pd.concat([enemy_df['Reference_Name'].value_counts(), team_df['Reference_Name'].value_counts()], axis = 1, sort = True, keys = ('Enemy Frequency', 'Team Frequency'))

,Enemy Frequency,Team Frequency
Androxus,31,31
Ash,24,23
Atlas,4,5
Barik,34,32
Bomb King,13,8
Buck,11,6
Cassie,2,7
Corvus,20,22
Dredge,17,16
Drogoz,11,6


## Whenever X is on my team, they are always mastery level 2

In [3]:
hero_dict = {}
for i in pop:
# Change all mentions of team in this line to enemy to get the enemy heroes with high/low levels
    hero_dict[i] = np.median(team_df_pop.loc[team_df_pop['Reference_Name'] == i]['Mastery_Level'])
    
# Change reverse to True to see who is always a high level on your team
sorted_by_level = [(k, v) for k, v in sorted(hero_dict.items(), key=lambda item: item[1], reverse = False)]

print("1st place: " + sorted_by_level[0][0])
print("Median mastery level: " + str(round(sorted_by_level[0][1], 0)))
print("----------")
print("2nd place: " + sorted_by_level[1][0])
print("Median mastery level: " + str(round(sorted_by_level[1][1], 0)))
print("----------")
print("3rd place: " + sorted_by_level[2][0])
print("Median mastery level: " + str(round(sorted_by_level[2][1], 0)))
print("----------")
print("4th place: " + sorted_by_level[3][0])
print("Median mastery level: " + str(round(sorted_by_level[3][1], 0)))
print("----------")
print("5th place: " + sorted_by_level[4][0])
print("Median mastery level: " + str(round(sorted_by_level[4][1], 0)))

1st place: Ruckus
Median mastery level: 24.0
----------
2nd place: Grohk
Median mastery level: 27.0
----------
3rd place: Lex
Median mastery level: 29.0
----------
4th place: Raum
Median mastery level: 34.0
----------
5th place: Barik
Median mastery level: 35.0


## We never win with X on our team

In [4]:
hero_dict = {}
for i in pop:
# Change all mentions of team in this line to enemy to get the enemy heroes with high/low win %
    hero_dict[i] = 100 * team_df_pop.loc[team_df_pop['Reference_Name'] == i]['Win_Status'].value_counts()['Winner'] / \
        team_df_pop['Reference_Name'].value_counts()[i]
    
# Change reverse to True to see who you always win with
sorted_by_percent = [(k, v) for k, v in sorted(hero_dict.items(), key=lambda item: item[1], reverse = False)]

print("1st place: " + sorted_by_percent[0][0])
print("Win %: " + str(round(sorted_by_percent[0][1], 1)))
print("----------")
print("2nd place: " + sorted_by_percent[1][0])
print("Win %: " + str(round(sorted_by_percent[1][1], 1)))
print("----------")
print("3rd place: " + sorted_by_percent[2][0])
print("Win %: " + str(round(sorted_by_percent[2][1], 1)))
print("----------")
print("4th place: " + sorted_by_percent[3][0])
print("Win %: " + str(round(sorted_by_percent[3][1], 1)))
print("----------")
print("5th place: " + sorted_by_percent[4][0])
print("Win %: " + str(round(sorted_by_percent[4][1], 1)))

1st place: Buck
Win %: 16.7
----------
2nd place: Makoa
Win %: 20.0
----------
3rd place: Kinessa
Win %: 21.1
----------
4th place: Lian
Win %: 29.4
----------
5th place: Evie
Win %: 30.8


## X is always good on their team, but bad on mine

In [5]:
hero_dict = {}
for i in pop:
    team_kda = np.median(
        (team_df_pop.loc[team_df_pop['Reference_Name'] == i]['Kills_Player'] + \
         0.5 * team_df_pop.loc[team_df_pop['Reference_Name'] == i]['Assists']) / \
        team_df_pop.loc[team_df_pop['Reference_Name'] == i]['Deaths']
    )
    enemy_kda = np.median(
        (enemy_df_pop.loc[enemy_df_pop['Reference_Name'] == i]['Kills_Player'] + \
         0.5 * enemy_df_pop.loc[enemy_df_pop['Reference_Name'] == i]['Assists']) / \
        enemy_df_pop.loc[enemy_df_pop['Reference_Name'] == i]['Deaths']
    )
    ratio = enemy_kda / team_kda
    hero_dict[i] = (ratio, team_kda, enemy_kda)
    
# Change reverse to False to see who is always good on your team but bad on theirs.
sorted_by_kda_ratio = [(k, v) for k, v in sorted(hero_dict.items(), key=lambda item: item[1], reverse = True)]

print("1st place: " + sorted_by_kda_ratio[0][0])
print("Median enemy KDA: " + str(round(sorted_by_kda_ratio[0][1][2], 1)))
print("Median team KDA: " + str(round(sorted_by_kda_ratio[0][1][1], 1)))
print("----------")
print("2nd place: " + sorted_by_kda_ratio[1][0])
print("Median enemy KDA: " + str(round(sorted_by_kda_ratio[1][1][2], 1)))
print("Median team KDA: " + str(round(sorted_by_kda_ratio[1][1][1], 1)))
print("----------")
print("3rd place: " + sorted_by_kda_ratio[2][0])
print("Median enemy KDA: " + str(round(sorted_by_kda_ratio[2][1][2], 1)))
print("Median team KDA: " + str(round(sorted_by_kda_ratio[2][1][1], 1)))
print("----------")
print("4th place: " + sorted_by_kda_ratio[3][0])
print("Median enemy KDA: " + str(round(sorted_by_kda_ratio[3][1][2], 1)))
print("Median team KDA: " + str(round(sorted_by_kda_ratio[3][1][1], 1)))
print("----------")
print("5th place: " + sorted_by_kda_ratio[4][0])
print("Median enemy KDA: " + str(round(sorted_by_kda_ratio[4][1][2], 1)))
print("Median team KDA: " + str(round(sorted_by_kda_ratio[4][1][1], 1)))

1st place: Makoa
Median enemy KDA: 1.3
Median team KDA: 0.9
----------
2nd place: Willo
Median enemy KDA: 1.8
Median team KDA: 1.3
----------
3rd place: Evie
Median enemy KDA: 1.2
Median team KDA: 0.9
----------
4th place: Grohk
Median enemy KDA: 2.2
Median team KDA: 1.7
----------
5th place: Ying
Median enemy KDA: 1.6
Median team KDA: 1.2


## X kills me all the time

In [6]:
hero_dict = {}
for hero in pop:
    match_ids = [j for j in enemy_df_pop.loc[enemy_df_pop['Reference_Name'] == hero]['Match']]

    match_counts = 0
    kill_sum = 0
    for match in match_list:

        for i in match:
            if i['playerName'] == 'Wakamar':
                win = int(i['Win_Status'] == 'Winner')
                match_counts = int(i['Match'] in match_ids)
                my_deaths = i['Deaths']

        if match_counts:
            total_team_deaths = 0
            for i in match:
                if int(i['Win_Status'] == 'Winner') == win:
                    total_team_deaths += i['Deaths']

                if i['Reference_Name'] == hero:
                    hero_kills = i['Kills_Player']

            kill_sum += my_deaths / total_team_deaths * hero_kills
            
    hero_dict[hero] = kill_sum
    
sorted_by_kills = [(k, v) for k, v in sorted(hero_dict.items(), key=lambda item: item[1], reverse = True)]

print("1st place: " + sorted_by_kills[0][0])
print("Estimated times they have killed me: " + str(round(sorted_by_kills[0][1], 0)))
print("----------")
print("2nd place: " + sorted_by_kills[1][0])
print("Estimated times they have killed me: " + str(round(sorted_by_kills[1][1], 0)))
print("----------")
print("3rd place: " + sorted_by_kills[2][0])
print("Estimated times they have killed me: " + str(round(sorted_by_kills[2][1], 0)))
print("----------")
print("4th place: " + sorted_by_kills[3][0])
print("Estimated times they have killed me: " + str(round(sorted_by_kills[3][1], 0)))
print("----------")
print("5th place: " + sorted_by_kills[4][0])
print("Estimated times they have killed me: " + str(round(sorted_by_kills[4][1], 0)))

1st place: Viktor
Estimated times they have killed me: 144.0
----------
2nd place: Vivian
Estimated times they have killed me: 101.0
----------
3rd place: Tyra
Estimated times they have killed me: 100.0
----------
4th place: Skye
Estimated times they have killed me: 96.0
----------
5th place: Androxus
Estimated times they have killed me: 80.0


## I kill X all the time

In [7]:
hero_dict = {}
for hero in pop:
    match_ids = [j for j in enemy_df_pop.loc[enemy_df_pop['Reference_Name'] == hero]['Match']]

    match_counts = 0
    kill_sum = 0
    for match in match_list:

        for i in match:
            if i['playerName'] == 'Wakamar':
                win = int(i['Win_Status'] == 'Winner')
                match_counts = int(i['Match'] in match_ids)
                my_kills = i['Kills_Player']

        if match_counts:
            total_team_kills = 0
            for i in match:
                if int(i['Win_Status'] == 'Winner') == win:
                    total_team_kills += i['Kills_Player']

                if i['Reference_Name'] == hero:
                    hero_deaths = i['Deaths']

            kill_sum += my_kills / total_team_kills * hero_deaths
            
    hero_dict[hero] = kill_sum
    
sorted_by_kills = [(k, v) for k, v in sorted(hero_dict.items(), key=lambda item: item[1], reverse = True)]

print("1st place: " + sorted_by_kills[0][0])
print("Estimated times I have killed them: " + str(round(sorted_by_kills[0][1], 0)))
print("----------")
print("2nd place: " + sorted_by_kills[1][0])
print("Estimated times I have killed them: " + str(round(sorted_by_kills[1][1], 0)))
print("----------")
print("3rd place: " + sorted_by_kills[2][0])
print("Estimated times I have killed them: " + str(round(sorted_by_kills[2][1], 0)))
print("----------")
print("4th place: " + sorted_by_kills[3][0])
print("Estimated times I have killed them: " + str(round(sorted_by_kills[3][1], 0)))
print("----------")
print("5th place: " + sorted_by_kills[4][0])
print("Estimated times I have killed them: " + str(round(sorted_by_kills[4][1], 0)))

1st place: Viktor
Estimated times I have killed them: 117.0
----------
2nd place: Seris
Estimated times I have killed them: 107.0
----------
3rd place: Tyra
Estimated times I have killed them: 103.0
----------
4th place: Raum
Estimated times I have killed them: 101.0
----------
5th place: Inara
Estimated times I have killed them: 75.0


## We never win on X map

In [8]:
team_df['Map_Game'] = team_df['Map_Game'].str.replace('Ranked', '')
team_df['Map_Game'] = team_df['Map_Game'].str.replace('LIVE', '')
map_pop = (team_df['Map_Game'].value_counts() > 20).keys()[team_df['Map_Game'].value_counts() > 20]

map_dict = {}
for i in map_pop:
    map_dict[i] = 100 * team_df.loc[team_df['Map_Game'] == i]['Win_Status'].value_counts()['Winner'] / \
        team_df['Map_Game'].value_counts()[i]
    
# Change reverse to True to see who you always win with
sorted_by_percent = [(k, v) for k, v in sorted(map_dict.items(), key=lambda item: item[1], reverse = False)]

print("1st place: " + sorted_by_percent[0][0])
print("Win %: " + str(round(sorted_by_percent[0][1], 1)))
print("----------")
print("2nd place: " + sorted_by_percent[1][0])
print("Win %: " + str(round(sorted_by_percent[1][1], 1)))
print("----------")
print("3rd place: " + sorted_by_percent[2][0])
print("Win %: " + str(round(sorted_by_percent[2][1], 1)))
print("----------")
print("4th place: " + sorted_by_percent[3][0])
print("Win %: " + str(round(sorted_by_percent[3][1], 1)))
print("----------")
print("5th place: " + sorted_by_percent[4][0])
print("Win %: " + str(round(sorted_by_percent[4][1], 1)))

1st place:  Splitstone Quarry
Win %: 27.6
----------
2nd place:  Ascension Peak
Win %: 33.3
----------
3rd place:  Frog Isle
Win %: 34.8
----------
4th place:  Shattered Desert
Win %: 40.0
----------
5th place:  Jaguar Falls
Win %: 41.2
